In [82]:
import pandas as pd
import numpy as np
import plotly_express as px

## Mungitura

In [83]:
df = pd.read_csv("DatasetUniti/Mungitura_old.csv")

In [84]:
df[['Minuti', 'Secondi']] = df['Durata (MM:SS)'].str.split(':', 1, expand=True)
df['Minuti'] = df['Minuti'].str.strip("0")
df['Secondi'] = df['Secondi'].str.strip("0")
df.loc[df["Minuti"] == '',"Minuti"] = "0"
df.loc[df["Secondi"] == '',"Secondi"] = "0"
df["MinutiTotali"] = df["Minuti"].astype("float") + df["Secondi"].astype("float")/60
mask = df["Ora inizio"] == "MANCANO I DATI DAL 17/04/2021 AL 12/02/2021 E NON SONO RECUPERABILI"
i = df[mask].index
df.drop(i,inplace=True)
df['Data'] = pd.to_datetime(df['Ora inizio']).dt.date
df['Tempo'] = pd.to_datetime(df['Ora inizio']).dt.time
df['Mese'] = pd.to_datetime(df['Ora inizio']).dt.month
df['Anno'] = pd.to_datetime(df['Ora inizio']).dt.year
df['Mese Anno'] = [str(x)[0:7] for x in df['Ora inizio']]

In [85]:
columns = ["Produzione (kg)","Flusso AS","Flusso AD","Flusso PS","Flusso PD",
           "Sangue AS","Sangue AD","Sangue PS","Sangue PD",
           "Conducibilità AS","Conducibilità AD","Conducibilità PS","Conducibilità PD",
           "PicchiFlusso AS","PicchiFlusso AD","PicchiFlusso PS","PicchiFlusso PD",
           'Produzione AS', 'Produzione AD','Produzione PS', 'Produzione PD'
]
df["Produzione PD"].str.replace(",",".").astype(float)
for column in columns:
    df[column] = df[column].astype(str).str.replace(",",".")
    df[column] = df[column].astype(float)

In [86]:
basi = ["Kick-off","Incompleto","Capezzoli non trovati","Capezzolo"]
cats = ["PD","AD","AS","PS","Tutto"]
for base in basi:
    df[base+"PD"]=0
    df[base+"AD"]=0
    df[base+"PS"]=0
    df[base+"AS"]=0
    col = base
    for _, i in df.iterrows():
        if str(i[col]) != "nan":
            #print(i[col]+" "+str(_))
            if str(i[col]) == "Tutto":
                #print("Tutto found "+str(_))
                df.loc[_,base+"AD"] = 1
                df.loc[_,base+"PD"] = 1
                df.loc[_,base+"AS"] = 1
                df.loc[_,base+"PS"] = 1
            else:
                if str(i[base]).find("AD") != -1:
                    #print("AD found "+str(_))
                    df.loc[_,base+"AD"] = 1
                if str(i[base]).find("PD")!= -1:
                    #print("PD found "+str(_))
                    df.loc[_,base+"PD"] = 1
                if str(i[base]).find("PS")!= -1:
                    #print("PS found "+str(_))
                    df.loc[_,base+"PS"] = 1
                if str(i[base]).find("AS")!= -1:
                    #print("AS found "+str(_))
                    df.loc[_,base+"AD"] = 1

In [87]:
df.to_csv("DatasetProcessed/new_mungitura_old.csv",index=False,sep=';',decimal=',')

## Traffico

In [105]:
df = pd.read_csv("DatasetUniti/traffico_animali_old.csv")

/var/folders/48/tzp62xjn58bb_nw1n30kx0sh0000gn/T/ipykernel_39504/1978675109.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("DatasetUniti/traffico_animali_old.csv")


In [106]:
col = "Tempo nell'area (hh:mm)"

df[['Ore', 'Minuti']] = df[col].str.split(':', 1, expand=True)
df['Minuti'] = df['Minuti'].str.strip("0")
df['Ore'] = df['Ore'].str.strip("0")
df.loc[df["Minuti"] == '',"Minuti"] = "0"
df.loc[df["Ore"] == '',"Ore"] = "0"
df["OreTotali"] = df["Ore"].astype("float") + df["Minuti"].astype("float")/60

In [107]:
df['Data'] = pd.to_datetime(df['Data/Ora']).dt.date
df['Tempo'] = pd.to_datetime(df['Data/Ora']).dt.time
df['Mese'] = pd.to_datetime(df['Data/Ora']).dt.month
df['Anno'] = pd.to_datetime(df['Data/Ora']).dt.year
df['Mese Anno'] = [str(x)[0:7] for x in df['Data']]

In [ ]:
mask = df["Risultato"].str.contains("kg",na=False)
df["Risultato"] = np.where(mask,df["Risultato"].str.replace(",","."),0)
df['Risultato'] = df['Risultato'].str.extract('([0-9][,.]*[0-9]*)')
df["Risultato"] = df["Risultato"].astype('float')

In [112]:
cats = ["PD","AD","AS","PS","Tutto"]
df["PD"]=0
df["AD"]=0
df["PS"]=0
df["AS"]=0
col = "Capezzoli non completamente munti"
for _, i in df.iterrows():
    if str(i[col]) != "nan":
        #print(i[col]+" "+str(_))
        if str(i[col]) == "Tutto":
            #print("Tutto found "+str(_))
            df.loc[_,"AD"] = 1
            df.loc[_,"PD"] = 1
            df.loc[_,"AS"] = 1
            df.loc[_,"PS"] = 1
        else:
            if str(i["Capezzoli non completamente munti"]).find("AD") != -1:
                #print("AD found "+str(_))
                df.loc[_,"AD"] = 1
            if str(i["Capezzoli non completamente munti"]).find("PD")!= -1:
                #print("PD found "+str(_))
                df.loc[_,"PD"] = 1
            if str(i["Capezzoli non completamente munti"]).find("PS")!= -1:
                #print("PS found "+str(_))
                df.loc[_,"PS"] = 1
            if str(i["Capezzoli non completamente munti"]).find("AS")!= -1:
                #print("AS found "+str(_))
                df.loc[_,"AD"] = 1

In [113]:
df.to_csv("DatasetProcessed/new_traffico_animali_old.csv",index=False,sep=';',decimal=',')

## Mungitura eventi

In [118]:
mungitura_eventi = pd.read_excel("DatasetUniti/Tutti gli eventi vecchio VMS.xls")

In [120]:
mungitura_eventi['Data'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.date
mungitura_eventi['Tempo'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.time
mungitura_eventi['Mese'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.month
mungitura_eventi['Anno'] = pd.to_datetime(mungitura_eventi['Data evento']).dt.year
mungitura_eventi['Mese Anno'] = [str(x)[0:7] for x in mungitura_eventi['Data']]

In [118]:
#mungitura_eventi["id"] = range(mungitura_eventi.shape[0])

In [119]:
#mungitura_eventi["count"] = mungitura_eventi.groupby(["Mese Anno"])["id"].transform("count")

In [121]:
mungitura_eventi.to_csv("DatasetProcessed/new_mungitura_eventi_old.csv",index=False,sep=';',decimal=',')